In [1]:
# Authentication to Google API
import os
import math
from collections import Counter
from google.cloud import vision
import re

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='imgtranslation_key.json'


In [15]:
from google.cloud import translate_v2 as translate
import json

def detect_text(path):
    """Detects text in the file."""

    client = vision.ImageAnnotatorClient()
    def save_json(data, filename):
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(data, file, indent=2, ensure_ascii=False)

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # for non-dense text 
    #response = client.text_detection(image=image)
    # for dense text
    translate_client = translate.Client()
    #response = client.document_text_detection(image=image)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    ocr_text = []
    output_data = []
    pattern = r'^[a-zA-Z0-9\s]+$'

    for text in texts[1:]:
        print(text.description)
        print(text.bounding_poly.vertices)
        if len(text.description.strip()) > 3:
            try:
                #detection = translate_client.detect_language(text.description)
                detection = translate_client.detect_language(text.description)
                source_language = detection['language']
                
                translation = translate_client.translate(text.description, target_language='en', source_language=source_language)
                
                line_data = {
                            "bounding_box": text.bounding_poly.vertices,
                            "original_text": text.description,
                            "translated_text": translation['translatedText']
                        }
                output_data.append(line_data)
            except Exception as e:
        # Handle the exception
                print(f"Exception occurred for item : {e}")
        # Continue to the next iteration of the loop
                continue
    print(output_data)
    output_json = json.dumps(output_data, indent=2).encode("utf-8")
    save_json(output_data, "output.json")    
    print(ocr_text)

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return ocr_text

In [16]:
image_path = r"C:\DL projects\GCP_imagetranslation\Test images\Thai test.png"
text = detect_text(image_path)

-
[x: 98
y: 34
, x: 104
y: 34
, x: 104
y: 36
, x: 98
y: 36
]
Reckitt
[x: 24
y: 26
, x: 92
y: 26
, x: 92
y: 43
, x: 24
y: 43
]
Key
[x: 113
y: 26
, x: 149
y: 26
, x: 149
y: 43
, x: 113
y: 43
]
Message
[x: 155
y: 26
, x: 241
y: 27
, x: 241
y: 45
, x: 155
y: 44
]
Exception occurred for item : 400 POST https://translation.googleapis.com/language/translate/v2?prettyPrint=false: Bad language pair: en|en [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'source', 'description': 'Source language: en'}, {'field': 'target', 'description': 'Target language: en'}]}]
Early
[x: 247
y: 27
, x: 296
y: 27
, x: 296
y: 44
, x: 247
y: 44
]
Exception occurred for item : 400 POST https://translation.googleapis.com/language/translate/v2?prettyPrint=false: Bad language pair: en|en [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'source', 'description': 'Source language: en'}, {'field': 'target', 'description': 'Target language: en'}]}]
L

TypeError: Object of type RepeatedCompositeContainer is not JSON serializable

In [ ]:
from google.cloud import translate_v2 as translate

# Initialize the translation client
translate_client = translate.Client()

# Text to detect language and translate
text = 'Hello, how are you? ...'

# Detect the language of the text
detection = translate_client.detect_language(text)
source_language = detection['language']

# Target language code
target_language = 'en'  # French

# Translate the text to the target language
if target_language=='en':
    translation=text
    translation = translate_client.translate(text, target_language=target_language, source_language=source_language)

# Print the original text, detected language, and translated text
print('Text:', text)
print('Detected Language:', source_language)
print('Translation:', translation['translatedText'])


In [ ]:
['\r\nเมื่อเติบโต ขอให้ควบคุม\nความรู้สึกของตัวเองให้ได้\nเมื่อเสียใจ ขอให้เรียกคืนสติ\nของตัวเองกลับมาให้เร็ว\nเมื่อทุกอย่างพังทลายลง\nขอให้รู้ว่าเราต้องใช้ชีวิตต่อไป\n지금\n- Weak-', '\r\nเมื่อ', '\r\nเติบโต', '\r\nขอ', '\r\nให้', '\r\nควบคุม', '\r\nความ', '\r\nรู้สึก', '\r\nของ', '\r\nตัว', '\r\nเอง', '\r\nให้', '\r\nได้', '\r\nเมื่อ', '\r\nเสียใจ', '\r\nขอ', '\r\nให้', '\r\nเรียก', '\r\nคืน', '\r\nสติ', '\r\nของ', '\r\nตัว', '\r\nเอง', '\r\nกลับ', '\r\nมา', '\r\nให้', '\r\nเร็ว', '\r\nเมื่อ', '\r\nทุก', '\r\nอย่าง', '\r\nพัง', '\r\nทลาย', '\r\nลง', '\r\nขอ', '\r\nให้', '\r\nรู้', '\r\nว่า', '\r\nเรา', '\r\nต้อง', '\r\nใช้', '\r\nชีวิต', '\r\nต่อ', '\r\nไป', '\r\n지금', '\r\n-', '\r\nWeak-']

In [51]:
import re

def has_only_text_and_numbers(input_str):
    # Regular expression pattern to match only text and numbers
    pattern = r'^[a-zA-Z0-9\s]+$'
    # Match the input string against the pattern
    if re.match(pattern, input_str):
        return True
    else:
        return False

# Test the function
input_str1 = "!"
input_str2 = "Hello@123"
print(has_only_text_and_numbers(input_str1))  # Output: True
print(has_only_text_and_numbers(input_str2))  # Output: False


False
False


In [61]:
from google.cloud import translate_v2 as translate
import json
import os
import math
from collections import Counter
from google.cloud import vision
import re

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='imgtranslation_key.json'

def detect_text(path):
    """Detects text in the file."""

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    translate_client = translate.Client()
    response = client.document_text_detection(image=image)
    #response = client.text_detection(image=image)
    #print(response)
    texts = response.text_annotations
    output_data = []
    pattern = r'^[a-zA-Z0-9\s]+$'
    test=[]

    for text in texts[1:]:
        if len(text.description.strip()) > 3:
            try:
                detection = translate_client.detect_language(text.description)
                source_language = detection['language']

                # Check if source_language is English ('en')
                if source_language != 'en':
                    translation = translate_client.translate(text.description, target_language='en', source_language=source_language)
                    x_values=[]
                    for vertex in text.bounding_poly.vertices:
                        x_values.append(vertex.x)
                        x_values.append(vertex.y)
                    #print(x_values)
                    #test.append(text.bounding_poly.vertices)
                    #print(test)
                    line_data = {
                                "bounding_box": x_values,
                                "original_text": text.description,
                                "translated_text": translation['translatedText']
                            }
                    output_data.append(line_data)
            except Exception as e:
                print(f"Exception occurred for item : {e}")
                continue
    print(output_data)
   
    json.dump(output_data, indent=2, ensure_ascii=False)

    return output_data


In [62]:
image_path = r"C:\DL projects\GCP_imagetranslation\Test images\german.jpg"
text = detect_text(image_path)

[{'bounding_box': [30, 51, 287, 51, 287, 100, 30, 100], 'original_text': 'Selbst', 'translated_text': 'Himself'}, {'bounding_box': [30, 150, 334, 150, 334, 216, 30, 216], 'original_text': 'längste', 'translated_text': 'longest'}, {'bounding_box': [29, 252, 355, 245, 356, 311, 30, 318], 'original_text': 'beginnt', 'translated_text': 'begins'}, {'bounding_box': [231, 354, 497, 358, 496, 410, 230, 406], 'original_text': 'ersten', 'translated_text': 'first'}, {'bounding_box': [30, 454, 338, 454, 338, 506, 30, 506], 'original_text': 'Schritt', 'translated_text': 'Step'}]


In [30]:
from google.cloud import translate_v2 as translate
import json

def detect_text(path):
    """Detects text in the file."""

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    translate_client = translate.Client()
    response = client.document_text_detection(image=image)
    #response = client.text_detection(image=image)
    #print(response)
    texts = response.text_annotations
    output_data = []
    pattern = r'^[a-zA-Z0-9\s]+$'
    test=[]

    for text in texts[1:]:
        if len(text.description.strip()) > 3:
            try:
                detection = translate_client.detect_language(text.description)
                source_language = detection['language']

                # Check if source_language is English ('en')
                if source_language != 'en':
                    translation = translate_client.translate(text.description, target_language='en', source_language=source_language)
                    x_values=[]
                    for vertex in text.bounding_poly.vertices:
                        x_values.append(vertex.x)
                        x_values.append(vertex.y)
                    #print(x_values)
                    #test.append(text.bounding_poly.vertices)
                    #print(test)
                    line_data = {
                                "bounding_box": x_values,
                                "original_text": text.description,
                                "translated_text": translation['translatedText']
                            }
                    output_data.append(line_data)
            except Exception as e:
                print(f"Exception occurred for item : {e}")
                continue
    print(output_data)
    # Serialize the output_data manually
    serialized_output = []
    for item in output_data:
        serialized_item = {
            "bounding_box": item["bounding_box"],
            "original_text": item["original_text"],
            "translated_text": item["translated_text"]
        }
        serialized_output.append(serialized_item)
   # print(serialized_output)
    
    # Save as JSON
    #with open("output.json", "w", encoding="utf-8") as file:
       # json.dump(serialized_output, file, indent=2, ensure_ascii=False)

    return serialized_output


X Values: ['29']
Y Values: ['50']


In [7]:
import re

bounding_boxes=[x: 30
y: 51
, x: 287
y: 51
, x: 287
y: 100
, x: 30
y: 100
]

# Extract numerical values using regular expressions
processed_boxes = []
for box in bounding_boxes:
    nums = re.findall(r'\d+', box)
    processed_boxes.extend(nums)

print(processed_boxes)


SyntaxError: invalid syntax (4082402891.py, line 3)

In [38]:
import argparse
from enum import Enum

In [39]:
class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5

In [41]:
def get_document_bounds(image_file, feature):
    """Finds the document bounds given an image and feature type.

    Args:
        image_file: path to the image file.
        feature: feature type to detect.

    Returns:
        List of coordinates for the corresponding feature type.
    """
    client = vision.ImageAnnotatorClient()

    bounds = []

    with open(image_file, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image)
    document = response.full_text_annotation

    # Collect specified feature bounds by enumerating all document features
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if feature == FeatureType.SYMBOL:
                            bounds.append(symbol.bounding_box)

                    if feature == FeatureType.WORD:
                        bounds.append(word.bounding_box)

                if feature == FeatureType.PARA:
                    bounds.append(paragraph.bounding_box)

            if feature == FeatureType.BLOCK:
                bounds.append(block.bounding_box)

    # The list `bounds` contains the coordinates of the bounding boxes.
    return bounds



In [43]:
bounds = get_document_bounds(r"C:\DL projects\GCP_imagetranslation\Test images\german.jpg", FeatureType.WORD)
print(bounds)

[vertices {
  x: 30
  y: 51
}
vertices {
  x: 287
  y: 51
}
vertices {
  x: 287
  y: 100
}
vertices {
  x: 30
  y: 100
}
, vertices {
  x: 314
  y: 51
}
vertices {
  x: 455
  y: 51
}
vertices {
  x: 455
  y: 100
}
vertices {
  x: 314
  y: 100
}
, vertices {
  x: 30
  y: 150
}
vertices {
  x: 334
  y: 150
}
vertices {
  x: 334
  y: 216
}
vertices {
  x: 30
  y: 216
}
, vertices {
  x: 358
  y: 150
}
vertices {
  x: 534
  y: 150
}
vertices {
  x: 534
  y: 216
}
vertices {
  x: 358
  y: 216
}
, vertices {
  x: 29
  y: 252
}
vertices {
  x: 355
  y: 245
}
vertices {
  x: 356
  y: 311
}
vertices {
  x: 30
  y: 318
}
, vertices {
  x: 378
  y: 245
}
vertices {
  x: 526
  y: 242
}
vertices {
  x: 527
  y: 307
}
vertices {
  x: 379
  y: 310
}
, vertices {
  x: 33
  y: 351
}
vertices {
  x: 197
  y: 353
}
vertices {
  x: 196
  y: 405
}
vertices {
  x: 32
  y: 403
}
, vertices {
  x: 231
  y: 354
}
vertices {
  x: 497
  y: 358
}
vertices {
  x: 496
  y: 410
}
vertices {
  x: 230
  y: 406
}
, ver

In [44]:
import re

data = """
vertices {
  x: 30
  y: 51
}
vertices {
  x: 287
  y: 51
}
vertices {
  x: 287
  y: 100
}
vertices {
  x: 30
  y: 100
}
, vertices {
  x: 314
  y: 51
}
vertices {
  x: 455
  y: 51
}
vertices {
  x: 455
  y: 100
}
vertices {
  x: 314
  y: 100
}
...
# More data here
"""

# Regular expression pattern to match 'x:' and 'y:' values
pattern = r'x:\s*(\d+)\s*y:\s*(\d+)'

# Extract x and y values using regular expression
matches = re.findall(pattern, data)

# Convert matches to list of tuples (x, y)
coordinates = [(int(x), int(y)) for x, y in matches]

print(coordinates)


[(30, 51), (287, 51), (287, 100), (30, 100), (314, 51), (455, 51), (455, 100), (314, 100)]
